In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

In [2]:
processed_df = pd.read_pickle('../data/preprocessed_df.pkl')

In [3]:
processed_df.head()

,created_utc,num_comments,subreddit,subreddit_subscribers,upvote_ratio,url,image?,total_text,tokenned_stopped_text,finalized_text
0,1.502177e+09,840.0,TheRedPill,0.0,0.65,https://www.reddit.com/r/TheRedPill/comments/6...,False,Google Fires Ph.D Biologist/Engineer For Claim...,"[google, fire, ph.d, biologist/engineer, claim...",google fire ph.d biologist/engineer claiming a...
1,1.480424e+09,759.0,TheRedPill,0.0,0.82,https://www.reddit.com/r/TheRedPill/comments/5...,False,HOW TO GET LAID LIKE A WARLORD: 37 Rules of Ap...,"[get, laid, like, warlord, 37, rule, approachi...",get laid like warlord 37 rule approaching mode...
2,1.502101e+09,179.0,TheRedPill,0.0,0.86,https://www.reddit.com/r/TheRedPill/comments/6...,False,I have been practicing Stoicism for 3 years no...,"[practicing, stoicism, 3, year, quality, life,...",practicing stoicism 3 year quality life increa...
3,1.433962e+09,954.0,TheRedPill,0.0,0.87,https://www.reddit.com/r/TheRedPill/comments/3...,False,"[META] Reddit rolls out first ban wave of ""Har...","[meta, reddit, roll, first, ban, wave, harassi...",meta reddit roll first ban wave harassing subr...
4,1.500270e+09,336.0,TheRedPill,0.0,0.90,https://www.reddit.com/r/TheRedPill/comments/6...,False,"Man gets his ex-GF to pay child support, and R...","[man, get, ex-gf, pay, child, support, reddit,...",man get ex-gf pay child support reddit freak o...


In [4]:
tfidf_vectorizer = TfidfVectorizer(max_df = .95)

text_transformed = tfidf_vectorizer.fit_transform(processed_df['finalized_text'])

In [5]:
text_transformed

<11009x27106 sparse matrix of type '<class 'numpy.float64'>'
	with 195800 stored elements in Compressed Sparse Row format>

In [6]:
k_clusterer = KMeans(n_clusters = 10, random_state = 305)

k_clusterer.fit(text_transformed)

KMeans(n_clusters=10, random_state=305)

In [7]:
processed_df['k_means_label'] = k_clusterer.labels_

In [8]:
processed_df['k_means_label'].value_counts()

1    7613
5    1519
2     443
6     422
9     261
3     207
0     202
8     190
7     104
4      48
Name: k_means_label, dtype: int64

In [9]:
processed_df

,created_utc,num_comments,subreddit,subreddit_subscribers,upvote_ratio,url,image?,total_text,tokenned_stopped_text,finalized_text,k_means_label
0,1.502177e+09,840.0,TheRedPill,0.0,0.65,https://www.reddit.com/r/TheRedPill/comments/6...,False,Google Fires Ph.D Biologist/Engineer For Claim...,"[google, fire, ph.d, biologist/engineer, claim...",google fire ph.d biologist/engineer claiming a...,2
1,1.480424e+09,759.0,TheRedPill,0.0,0.82,https://www.reddit.com/r/TheRedPill/comments/5...,False,HOW TO GET LAID LIKE A WARLORD: 37 Rules of Ap...,"[get, laid, like, warlord, 37, rule, approachi...",get laid like warlord 37 rule approaching mode...,2
2,1.502101e+09,179.0,TheRedPill,0.0,0.86,https://www.reddit.com/r/TheRedPill/comments/6...,False,I have been practicing Stoicism for 3 years no...,"[practicing, stoicism, 3, year, quality, life,...",practicing stoicism 3 year quality life increa...,5
3,1.433962e+09,954.0,TheRedPill,0.0,0.87,https://www.reddit.com/r/TheRedPill/comments/3...,False,"[META] Reddit rolls out first ban wave of ""Har...","[meta, reddit, roll, first, ban, wave, harassi...",meta reddit roll first ban wave harassing subr...,0
4,1.500270e+09,336.0,TheRedPill,0.0,0.90,https://www.reddit.com/r/TheRedPill/comments/6...,False,"Man gets his ex-GF to pay child support, and R...","[man, get, ex-gf, pay, child, support, reddit,...",man get ex-gf pay child support reddit freak o...,2
...,...,...,...,...,...,...,...,...,...,...,...
11004,1.633457e+09,900.0,Anarcho_Capitalism,186148.0,0.88,https://v.redd.it/0ptsjdwh9or71,False,"The is Los Angeles, California","[los, angeles, california]",los angeles california,1
11005,1.632157e+09,2229.0,Anarcho_Capitalism,186148.0,0.83,https://i.redd.it/xmgcuoawtoo71.png,True,Personal freedoms,"[personal, freedom]",personal freedom,1
11006,1.591131e+09,141.0,Anarcho_Capitalism,186148.0,0.93,https://i.imgur.com/ticpuuy.jpg,True,Progressive Liberal,"[progressive, liberal]",progressive liberal,1
11007,1.638208e+09,506.0,Anarcho_Capitalism,186148.0,0.89,https://i.redd.it/dcyjbpirmk281.jpg,True,Welp…,[welp…],welp…,1


In [10]:
agg_clusterer = AgglomerativeClustering(n_clusters = 100, compute_full_tree = True, compute_distances = True)
agg_clusterer.fit(text_transformed.toarray())

AgglomerativeClustering(compute_distances=True, compute_full_tree=True,
                        n_clusters=100)

In [11]:
processed_df['agg_labels'] = agg_clusterer.labels_

In [12]:
processed_df['agg_labels'].value_counts()

3     7883
2      350
22     325
15     201
9      115
      ... 
65       7
29       7
51       7
91       6
84       6
Name: agg_labels, Length: 100, dtype: int64

In [14]:
processed_df.loc[processed_df['agg_labels'] == 2]

,created_utc,num_comments,subreddit,subreddit_subscribers,upvote_ratio,url,image?,total_text,tokenned_stopped_text,finalized_text,k_means_label,agg_labels
5,1.452025e+09,926.0,TheRedPill,0.0,0.88,https://www.reddit.com/r/TheRedPill/comments/3...,False,"1000 migrants in Germany sexually assault, rob...","[1000, migrant, germany, sexually, assault, ro...",1000 migrant germany sexually assault rob rape...,2,2
8,1.503981e+09,364.0,TheRedPill,0.0,0.88,https://www.reddit.com/r/TheRedPill/comments/6...,False,Three ways to consciously manipulate women bef...,"[three, way, consciously, manipulate, woman, s...",three way consciously manipulate woman subcons...,2,2
23,1.537322e+09,232.0,TheRedPill,0.0,0.82,https://www.reddit.com/r/TheRedPill/comments/9...,False,Vagina EnvyWe have a lot of posts rightfully s...,"[vagina, envywe, lot, post, rightfully, scoffi...",vagina envywe lot post rightfully scoffing chi...,2,2
36,1.426861e+09,477.0,TheRedPill,0.0,0.81,https://www.reddit.com/r/TheRedPill/comments/2...,False,The red pill are a bunch of bigots who can't g...,"[red, pill, bunch, bigot, ca, get, laid.they, ...",red pill bunch bigot ca get laid.they virgin n...,2,2
48,1.455796e+09,508.0,TheRedPill,0.0,0.84,https://www.reddit.com/r/TheRedPill/comments/4...,False,"Calling your wife/girlfriend fat, now official...","[calling, wife/girlfriend, fat, officially, do...",calling wife/girlfriend fat officially domesti...,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...
10753,1.619284e+09,142.0,EnoughCommieSpam,61310.0,0.97,https://i.redd.it/kaw251d705v61.jpg,True,"Press ""X"" to doubt","[press, x, doubt]",press x doubt,1,2
10880,1.603098e+09,82.0,TheLeftCantMeme,40481.0,0.94,https://i.redd.it/tx3bfhn27wt51.jpg,True,Yeah.....a false rape accusation can do that t...,"[yeah, ....., false, rape, accusation, man]",yeah ..... false rape accusation man,1,2
10905,1.610463e+09,93.0,TheLeftCantMeme,40481.0,0.81,https://i.redd.it/qxf3bgdszwa61.jpg,True,Average post on r/murderedbyAOC,"[average, r/murderedbyaoc]",average r/murderedbyaoc,1,2
10940,1.645758e+09,356.0,Anarcho_Capitalism,186148.0,0.94,https://v.redd.it/qh11unrdawj81,False,Don't Thread On Me Rus !!!,"[thread, ru]",thread ru,1,2
